In [ ]:
### SVM

In [46]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from matplotlib import pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.model_selection import train_test_split

In [47]:
df = pd.read_csv("/home/mete/Workspace/Okul/Ders/Bil479/Data/train.csv")
df_test = pd.read_csv("/home/mete/Workspace/Okul/Ders/Bil479/Data/test.csv") # hedef sütunu içermiyor
df_correct = pd.read_csv("/home/mete/Workspace/Okul/Ders/Bil479/Data/testtarget.csv")

In [48]:
df = df.fillna(0)
df_test = df_test.fillna(0)

In [49]:
corr = df.corr()
high_correlated_columns = corr[abs(corr["SalePrice"])>0.2].index
high_correlated_columns = high_correlated_columns.drop("SalePrice")

In [50]:
x_train = df[high_correlated_columns].values
x_test = df_test[high_correlated_columns].values

In [51]:
y_test = df_correct["SalePrice"]
y_train = df["SalePrice"]

In [52]:
y_train_max = max(y_train.values)
y_train_min = min(y_train.values)
y_train = (y_train-y_train_min)/(y_train_max-y_train_min)

In [53]:
y_test_max = max(y_test.values)
y_test_min = min(y_test.values)
y_test = (y_test-y_test_min)/(y_test_max-y_test_min)

In [54]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

x_train = min_max_scaler.fit_transform(x_train)
x_test = min_max_scaler.fit_transform(x_test)

In [55]:
y_train = np.array([int((x/(np.std(y_train)*2))) for x in y_train])
y_test = np.array([int((x/(np.std(y_test)*2))) for x in y_test])

In [92]:
clf = svm.SVC(kernel="poly",degree=7,gamma="scale")
clf.fit(x_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=7, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [93]:
y_pred = clf.predict(x_test)

In [94]:
conf_matrix = confusion_matrix(y_test,y_pred)

In [95]:
print(conf_matrix)

[[792  60   0   0]
 [ 83 431  15   0]
 [  6  20  43   0]
 [  0   0   9   0]]


In [96]:
conf_vals = []
for i in range(conf_matrix.shape[0]):
    TP = conf_matrix[i,i]
    FP = conf_matrix[i,:].sum() - TP
    FN = conf_matrix[:,i].sum() - TP
    TN = conf_matrix.sum() - (TP + FP + FN)
    conf_vals.append([TP,FP,FN,TN])
conf_vals = np.array(conf_vals)

In [97]:
# Total Accuracy = (TP + TN)/(TP + TN + FP + FN)
acc = (y_test == y_pred).sum()/y_test.shape[0]
print(f"Accuracy:{acc}")

Accuracy:0.8677176148046607


In [98]:
# Accuracy = (TP + TN)/(TP + TN + FP + FN)
for i in range(conf_matrix.shape[0]):
    s = (conf_vals[i,0] + conf_vals[i,3]) / conf_vals[i,:].sum()
    print(f"Sınıf{i} = {s}")

Sınıf0 = 0.8978752570253599
Sınıf1 = 0.8779986291980809
Sınıf2 = 0.9657299520219328
Sınıf3 = 0.9938313913639479


In [99]:
print("Sensitivity------------------------------------")
# Sensitivity = TP / TP + FN
sm = 0
for i in range(conf_matrix.shape[0]):
    s = conf_vals[i,0] / (conf_vals[i,0] + conf_vals[i,2])
    if not (np.isnan(s)):
        sm += s
        
    print(f"Sınıf{i} = TP({conf_vals[i,0]}) / (TP({conf_vals[i,0]}) + FN({conf_vals[i,2]})) = {s}")
print(f"avg = {sm/conf_matrix.shape[0]}")   

Sınıf0 = TP(792) / (TP(792) + FN(89)) = 0.8989784335981839
Sınıf1 = TP(431) / (TP(431) + FN(80)) = 0.8434442270058709
Sınıf2 = TP(43) / (TP(43) + FN(24)) = 0.6417910447761194
Sınıf3 = TP(0) / (TP(0) + FN(0)) = nan
avg = 0.5960534263450435


/home/mete/Workspace/Okul/main_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in long_scalars
  after removing the cwd from sys.path.


In [91]:
print("Specificity-----------------------------------")
# Specificity = TN / TN + FP
sm = 0
for i in range(conf_matrix.shape[0]):
    s = conf_vals[i,3] / (conf_vals[i,3] + conf_vals[i,1])
    if not (np.isnan(s)):
        sm += s
    print(f"Sınıf{i} = TN({conf_vals[i,3]}) / (TN({conf_vals[i,3]})+FP({conf_vals[i,1]})) = {s}")
print(f"avg = {sm/conf_matrix.shape[0]}") 


Sınıf0 = TN(506) / (TN(506)+FP(38)) = 0.9301470588235294
Sınıf1 = TN(874) / (TN(874)+FP(113)) = 0.8855116514690983
Sınıf2 = TN(1364) / (TN(1364)+FP(23)) = 0.9834174477289113
Sınıf3 = TN(1450) / (TN(1450)+FP(9)) = 0.9938313913639479


In [82]:
print("Precision-----------------------------------")
##### Precision = TP / TP + FP
sm = 0
for i in range(conf_matrix.shape[0]):
    s = conf_vals[i,0] / (conf_vals[i,0] + conf_vals[i,1])
    if not (np.isnan(s)):
        sm += s
    print(f"Sınıf{i} = TP({conf_vals[i,0]}) / (TP({conf_vals[i,0]}) + FP({conf_vals[i,1]})) = {s}")
print(f"avg = {sm/conf_matrix.shape[0]}") 

Sınıf0 = TP(830) / (TP(830) + FP(22)) = 0.9741784037558685
Sınıf1 = TP(398) / (TP(398) + FP(131)) = 0.7523629489603024
Sınıf2 = TP(34) / (TP(34) + FP(35)) = 0.4927536231884058
Sınıf3 = TP(0) / (TP(0) + FP(9)) = 0.0
